# 프론트엔드에 Browser Live View 임베드하기

## 개요

Amazon DCV (NICE DCV)와 AWS Bedrock AgentCore Browser를 사용하여 웹 애플리케이션에 실시간 브라우저 스트리밍과 Browser의 인간 제어 기능을 통합하는 방법을 배웁니다.

### 튜토리얼 세부 정보

| 정보                | 세부 사항                                                                          |
|:--------------------|:---------------------------------------------------------------------------------|
| 튜토리얼 유형        | Integration Guide                                                                |
| Agent 유형          | Single                                                                           |
| Agentic Framework   | Nova Act                                                                         |
| LLM model           | Amazon Nova Act model                                                            |
| 튜토리얼 구성 요소   | DCV live streaming, AgentCore Browser, WebSocket authentication                 |
| 튜토리얼 분야        | Web Development                                                                  |
| 예제 복잡도          | Medium                                                                           |
| 사용된 SDK          | Amazon BedrockAgentCore Python SDK, Nova Act, Amazon DCV Web Client SDK          |

### 튜토리얼 아키텍처

이 튜토리얼은 AWS Bedrock AgentCore Browser와 Amazon DCV 프로토콜을 사용하여 실시간으로 라이브 브라우저 세션을 스트리밍하는 방법을 보여줍니다.

### 튜토리얼 주요 기능

* headless 방식으로 browser tool 사용하기
* browser tool과 함께 Nova Act 사용하기
* React 애플리케이션에 browser live view 임베드하기

## 사전 요구 사항

이 튜토리얼을 실행하려면 다음이 필요합니다:

* Python 3.10+
* AWS 자격 증명 구성. IAM role/user는 다음 권한이 있어야 합니다 https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/browser-onboarding.html#browser-credentials-config
* Amazon Bedrock AgentCore SDK
* HTML/JavaScript 기본 지식

### 필수 Python 패키지 설치

In [ ]:
# 필수 패키지 설치
!pip install bedrock-agentcore boto3 -q

## DCVjs SDK 다운로드 및 설정

DCVjs SDK는 애플리케이션에서 실시간 브라우저 스트리밍을 가능하게 합니다.

### 수행할 작업:

1. **자동으로 다운로드** AWS CloudFront에서 최신 DCVjs SDK 다운로드
2. **추출 및 구성** 올바른 디렉토리 구조로 파일 정리
3. **설치 확인** 모든 것이 준비되었는지 확인

SDK는 다음과 같이 구성됩니다:

```
dcv-sdk/
├── dcvjs-umd/          # ← 이 버전을 사용합니다
│   ├── dcv.js
│   └── dcv/            # 비디오 디코딩을 위한 Worker 파일
├── dcvjs-esm/          # ES Module 버전
└── dcv-ui/             # 선택적 UI 컴포넌트
```

### 중요 사항:

- SDK는 정적 파일로 제공되어야 합니다 (npm 패키지로 import하지 않음)
- Worker 파일은 비디오 디코딩을 위해 런타임에 접근 가능해야 합니다
- 설치를 자동으로 확인합니다

**아래 셀을 실행하여 SDK를 자동으로 다운로드하고 설정하세요.**

In [ ]:
import urllib.request
import zipfile
import os
import shutil
from pathlib import Path

# DCV SDK 설정
DCV_SDK_URL = "https://d1uj6qtbmh3dt5.cloudfront.net/webclientsdk/nice-dcv-web-client-sdk-1.9.100-952.zip"
DCV_SDK_DIR = "dcv-sdk"
ZIP_FILE = "dcv-sdk.zip"

def download_dcv_sdk():
    """CloudFront에서 DCV SDK 다운로드"""
    print("📦 Downloading DCV SDK...")
    print(f"   Source: {DCV_SDK_URL}")
    
    # 진행률과 함께 다운로드
    def report_progress(block_num, block_size, total_size):
        downloaded = block_num * block_size
        percent = min(downloaded * 100 / total_size, 100)
        print(f"\r   Progress: {percent:.1f}%", end='', flush=True)
    
    urllib.request.urlretrieve(DCV_SDK_URL, ZIP_FILE, report_progress)
    print()  # 진행률 후 새 줄
    print("✅ Download complete!")

def extract_dcv_sdk():
    """DCV SDK를 dcv-sdk 디렉토리로 추출"""
    print("\n📂 Extracting DCV SDK...")
    
    # 기존 디렉토리가 있으면 제거
    if os.path.exists(DCV_SDK_DIR):
        shutil.rmtree(DCV_SDK_DIR)
        print("   Removed old dcv-sdk directory")
    
    # 임시 추출 디렉토리 생성
    temp_dir = "dcv-sdk-temp"
    if os.path.exists(temp_dir):
        shutil.rmtree(temp_dir)
    
    # zip 파일을 임시 디렉토리로 추출
    with zipfile.ZipFile(ZIP_FILE, 'r') as zip_ref:
        zip_ref.extractall(temp_dir)
    
    print("   Extracted to temporary directory")
    
    # 실제 SDK 디렉토리 찾기 (내부에 중첩되어 있음)
    # 구조: dcv-sdk-temp/nice-dcv-web-client-sdk/[실제 파일들]
    nested_dir = os.path.join(temp_dir, "nice-dcv-web-client-sdk")
    
    if os.path.exists(nested_dir):
        # 중첩된 디렉토리를 대상 위치로 이동
        shutil.move(nested_dir, DCV_SDK_DIR)
        print("   Moved SDK files to dcv-sdk/")
    else:
        # 구조가 다르면 temp를 target으로 이름 변경
        shutil.move(temp_dir, DCV_SDK_DIR)
        print("   Organized SDK files")
    
    # 정리
    if os.path.exists(temp_dir):
        shutil.rmtree(temp_dir)
    
    os.remove(ZIP_FILE)
    print("✅ Extraction complete!")
    print("   Cleaned up temporary files")

def verify_installation():
    """필요한 모든 파일이 제자리에 있는지 확인"""
    print("\n🔍 Verifying installation...")
    
    required_files = [
        "dcv-sdk/dcvjs-umd/dcv.js",
        "dcv-sdk/dcvjs-umd/dcv",  # 디렉토리
        "dcv-sdk/dcvjs-esm/dcv.js",
    ]
    
    all_good = True
    for file_path in required_files:
        if os.path.exists(file_path):
            file_type = "📁" if os.path.isdir(file_path) else "📄"
            print(f"   {file_type} {file_path} ✓")
        else:
            print(f"   ❌ {file_path} - NOT FOUND")
            all_good = False
    
    if all_good:
        print("\n✅ All required files are in place!")
        
        # worker 파일 개수 세기
        worker_dir = Path("dcv-sdk/dcvjs-umd/dcv")
        if worker_dir.exists():
            worker_files = list(worker_dir.glob('*.js')) + list(worker_dir.glob('*.wasm'))
            print(f"   Found {len(worker_files)} worker files for video decoding")
        
        # 파일 크기 표시
        dcv_js = Path("dcv-sdk/dcvjs-umd/dcv.js")
        if dcv_js.exists():
            size_mb = dcv_js.stat().st_size / (1024 * 1024)
            print(f"   Main SDK file size: {size_mb:.2f} MB")
        
        return True
    else:
        print("\n❌ Some files are missing. Please check the extraction.")
        
        # 실제로 가지고 있는 것 표시
        print("\n🔍 Debugging - Current structure:")
        if os.path.exists(DCV_SDK_DIR):
            for root, dirs, files in os.walk(DCV_SDK_DIR):
                level = root.replace(DCV_SDK_DIR, '').count(os.sep)
                indent = ' ' * 2 * level
                print(f"{indent}{os.path.basename(root)}/")
                sub_indent = ' ' * 2 * (level + 1)
                for file in files[:5]:  # 처음 5개 파일 표시
                    print(f"{sub_indent}{file}")
                if len(files) > 5:
                    print(f"{sub_indent}... and {len(files) - 5} more files")
                if level > 2:  # 깊이 제한
                    break
        return False

def show_directory_structure():
    """디렉토리 구조 표시"""
    print("\n📋 Directory Structure:")
    print("="*60)
    
    def print_tree(directory, prefix="", max_depth=3, current_depth=0):
        if current_depth >= max_depth:
            return
        
        try:
            contents = sorted(Path(directory).iterdir(), key=lambda x: (not x.is_dir(), x.name))
            for i, path in enumerate(contents):
                is_last = i == len(contents) - 1
                current_prefix = "└── " if is_last else "├── "
                
                # 중요한 파일의 파일 크기 표시
                size_info = ""
                if path.is_file() and path.suffix in ['.js', '.wasm']:
                    size_mb = path.stat().st_size / (1024 * 1024)
                    if size_mb > 0.1:
                        size_info = f" ({size_mb:.1f}MB)"
                
                print(f"{prefix}{current_prefix}{path.name}{'/' if path.is_dir() else ''}{size_info}")
                
                if path.is_dir() and current_depth < max_depth - 1:
                    extension = "    " if is_last else "│   "
                    print_tree(path, prefix + extension, max_depth, current_depth + 1)
        except PermissionError:
            pass
    
    print_tree(DCV_SDK_DIR)
    print("="*60)

# 메인 실행
try:
    # 이미 설치되어 있는지 확인
    if os.path.exists("dcv-sdk/dcvjs-umd/dcv.js"):
        print("⚠️  DCV SDK already exists!")
        print("   Current installation is valid.")
        
        # 빠른 확인
        if verify_installation():
            show_directory_structure()
            print("\n✅ Existing DCV SDK is ready to use!")
            print("\nℹ️  To re-download, delete the 'dcv-sdk' directory first.")
    else:
        # 다운로드 및 설정
        download_dcv_sdk()
        extract_dcv_sdk()
    
    if verify_installation():
        show_directory_structure()
        print("\n🎉 DCV SDK is ready to use!")
        print("\nℹ️  The SDK is now available at: ./dcv-sdk/")
        print("   - UMD version: dcv-sdk/dcvjs-umd/dcv.js")
        print("   - ESM version: dcv-sdk/dcvjs-esm/dcv.js")
        print("   - UI components: dcv-sdk/dcv-ui/")
    else:
        print("\n⚠️  Installation incomplete. Please check for errors above.")
        
except KeyboardInterrupt:
    print("\n\n⚠️  Download cancelled by user")
except Exception as e:
    print(f"\n❌ Error during installation: {e}")
    import traceback
    traceback.print_exc()

## 아키텍처 흐름

DCVjs client SDK가 라이브 브라우저를 스트리밍하는 방법:

```
┌──────────────────────┐
│   Python Backend     │
│ (AgentCore Browser)  │
└──────────┬───────────┘
           │
           │ 1. 브라우저 세션 생성
           │ 2. presigned URL 생성
           ↓
┌──────────────────────┐
│  DCV Streaming       │
│  Server              │
└──────────┬───────────┘
           │
           │ 3. DCV 프로토콜을 통한 스트리밍
           ↓
┌──────────────────────┐
│   Frontend           │
│ (DCVjs SDK)          │
│ - 인증               │
│ - 스트림 렌더링      │
└──────────────────────┘
```

**흐름 단계:**

1. AgentCore Browser Tool (Backend)이 브라우저 세션 생성
2. DCV Streaming Server가 스트리밍 URL 및 인증 토큰 생성
3. 인간 제어를 처리하기 위한 API 서버 생성. 인간 제어는 update_browser_stream API를 사용합니다 (https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/browser-update-stream.html).
4. DCVjs SDK (Frontend)가 연결하고 스트림 렌더링
5. Frontend가 인간 제어를 구현. 이는 인간이 Browser 작업을 처리하고 Agent가 아닌 경우에 유용합니다. 그런 다음 Agent에게 제어권을 다시 줄 수 있습니다.
4. HTML이 실시간 브라우저 뷰 표시

## Step 0: 환경 설정

먼저 필요한 라이브러리를 import하고 AWS 구성을 설정합니다.

In [ ]:
from bedrock_agentcore.tools.browser_client import BrowserClient
import boto3
import json

# boto3 session에서 AWS region 가져오기
session = boto3.Session()
region = session.region_name

print(f"Using AWS Region: {region}")
print("Environment setup complete!")

## Step 1: Browser Control API Server 생성

Human Take Over 기능을 지원하기 위해 브라우저 제어 API 호출을 처리할 간단한 API 서버를 생성해야 합니다. 이 서버는 automation mode를 활성화 및 비활성화하기 위한 엔드포인트를 제공합니다.

FastAPI를 사용하여 이 API 서버를 구현해 보겠습니다:

In [ ]:
from fastapi import FastAPI, HTTPException
from fastapi.responses import JSONResponse
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
import uvicorn
import threading

class AutomationStreamUpdate(BaseModel):
    streamStatus: str  # 'enabled' 또는 'disabled'

class BrowserStreamUpdateRequest(BaseModel):
    automationStreamUpdate: AutomationStreamUpdate

class BrowserControlServer:
    """브라우저 제어 API 엔드포인트를 처리하는 서버"""
    
    def __init__(self, browser_client, port=8081):
        """제어 서버 초기화"""
        self.browser_client = browser_client
        self.port = port
        self.app = FastAPI(title="Browser Control API")
        self.server_thread = None
        self.is_running = False
        
        # CORS middleware 추가 - cross-origin 요청 허용
        self.app.add_middleware(
            CORSMiddleware,
            allow_origins=["*"],  # 데모 목적으로 모든 origin 허용
            allow_credentials=True,
            allow_methods=["*"],
            allow_headers=["*"],
        )
        
        # 라우트 설정
        self._setup_routes()
    
    def _setup_routes(self):
        """FastAPI 라우트 설정"""
        
        @self.app.get("/")
        async def root():
            """루트 엔드포인트"""
            return {"message": "Browser Control API is running."}
        
        @self.app.post("/api/update_browser_stream")
        async def update_browser_stream(request: BrowserStreamUpdateRequest):
            """브라우저 스트림 automation 상태 업데이트"""
            try:
                stream_status = request.automationStreamUpdate.streamStatus
                print(f"Updating browser stream status to: {stream_status}")
                
                if stream_status == "disabled":
                    # 제어권 가져오기 (automation 비활성화)
                    self.browser_client.take_control()
                    print("✅ Human control enabled (automation disabled)")
                    return {"status": "success", "message": "Human control enabled"}
                
                elif stream_status == "enabled":
                    # 제어권 반환 (automation 활성화)
                    self.browser_client.release_control()
                    print("✅ Automation enabled")
                    return {"status": "success", "message": "Automation enabled"}
                    
                else:
                    print(f"❌ Invalid stream status: {stream_status}")
                    return JSONResponse(
                        status_code=400,
                        content={"status": "error", "message": f"Invalid stream status: {stream_status}. Must be 'enabled' or 'disabled'"}
                    )
                    
            except Exception as e:
                print(f"❌ Error updating browser stream: {e}")
                return JSONResponse(
                    status_code=500,
                    content={"status": "error", "message": str(e)}
                )
    
    def start(self):
        """API 서버 시작"""
        # 스레드에서 서버 실행
        def run_server():
            uvicorn.run(self.app, host="0.0.0.0", port=self.port, log_level="error")
        
        self.server_thread = threading.Thread(target=run_server, daemon=True)
        self.server_thread.start()
        self.is_running = True
        
        print(f"🌐 Browser Control API server running at: http://localhost:{self.port}")
        print(f"🔓 CORS enabled: Allowing requests from all origins")
        return f"http://localhost:{self.port}"

# 참고: control server는 browser_client 생성 후 Step 1에서 초기화됩니다
print("✅ Browser Control Server class defined")
print("The server will be initialized after the browser client is created.")

## Step 2: Browser 세션 생성 및 스트리밍 URL 가져오기

AgentCore Browser client를 초기화하고 프론트엔드 스트리밍에 필요한 URL을 생성합니다.

In [ ]:
# BrowserClient 생성 및 세션 시작
browser_client = BrowserClient(region)
browser_client.start()

print("✅ Browser session started")


### Step 2.1 스트리밍 URL 가져오기

아래 코드를 실행하여 라이브 스트리밍 presigned URL을 가져옵니다.

In [ ]:
# Agent가 연결할 automation endpoint websocket URL과 signed headers 가져오기
ws_url, headers = browser_client.generate_ws_headers()
print(f"\n📡 WebSocket URL: {ws_url[:60]}...")
print(f"🔐 Headers configured: {list(headers.keys())}")

# 5분 후 만료되는 presigned live-view URL 가져오기
live_view_url = browser_client.generate_live_view_url(expires=300)
print(f"\n🔗 Live View URL generated (expires in 300 seconds)")
print(f"   URL: {live_view_url[:80]}...")

# 브라우저 제어 서버 생성 및 시작
control_server = BrowserControlServer(browser_client, port=8081)
control_api_url = control_server.start()
print(f"\n🌐 Control API server running at: {control_api_url}")
print(f"   API endpoint: {control_api_url}/api/update_browser_stream")

# 프론트엔드를 위해 저장
session_data = {
    "presignedUrl": live_view_url,
    "sessionId": "demo-session",
    "authToken": "demo-session"
}

print("\n✅ Ready to connect from frontend!")

## Step 3: Live View 생성 및 실행

이 단계는 자동으로 다음을 수행합니다:

✅ DCV 통합이 포함된 HTML 페이지 생성

✅ presigned URL 자동 주입

✅ 브라우저 제어를 위한 Human Take Over 버튼 추가

✅ 해상도 선택 컨트롤 추가 (720p, 900p, 1080p, 1440p)

✅ 필요한 모든 파일이 제자리에 있는지 확인

✅ 로컬 웹 서버 시작 (포트 8080)

✅ 브라우저 자동 열기

### 표시될 내용:

1. 실제 presigned URL이 포함된 HTML 파일 생성
2. 파일 구조 확인
3. 웹 서버 시작
4. 라이브 브라우저 스트림을 보여주는 브라우저 열기

### 뷰어의 기능:

- DCV를 통한 실시간 브라우저 스트리밍
- 브라우저를 수동으로 제어하기 위한 Human Take Over 버튼
- 디스플레이 크기를 조정하기 위한 해상도 컨트롤 (720p ~ 1440p)
- 상태 표시기 (Loading → Authenticating → Connected)
- 자동 오류 처리
- 정보 패널이 있는 깔끔한 UI

**참고:** 웹 서버는 계속 실행됩니다. 중지하려면 중지 버튼(■)을 클릭하거나 커널을 재시작하세요.

In [ ]:
import http.server
import socketserver
import threading
import webbrowser
import time
import os
from pathlib import Path

# 설정
HTML_FILE = "browser_live_view.html"
SERVER_PORT = 8080
SERVER_URL = f"http://localhost:{SERVER_PORT}/{HTML_FILE}"
CONTROL_API_URL = "http://localhost:8081"  # control API 서버의 URL

def create_html_with_url(presigned_url, session_id="demo-session"):
    """presigned URL이 이미 주입된 HTML 파일 생성"""
    print("📝 Creating HTML file with live view integration...")

    html_content = f'''<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Browser Live View - AWS AgentCore</title>
    <style>
        * {{
            margin: 0;
            padding: 0;
            box-sizing: border-box;
        }}
        body {{
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            min-height: 100vh;
            padding: 20px;
        }}
        .container {{
            max-width: 1600px;
            margin: 0 auto;
        }}
        .header {{
            background: white;
            padding: 20px 30px;
            border-radius: 10px;
            margin-bottom: 20px;
            box-shadow: 0 4px 6px rgba(0,0,0,0.1);
        }}
        h1 {{
            color: #333;
            font-size: 28px;
            margin-bottom: 10px;
        }}
        .subtitle {{
            color: #666;
            font-size: 14px;
        }}
        .status {{
            padding: 15px 20px;
            margin-bottom: 20px;
            border-radius: 8px;
            font-weight: 600;
            display: flex;
            align-items: center;
            gap: 10px;
            box-shadow: 0 2px 4px rgba(0,0,0,0.1);
            transition: all 0.3s ease;
        }}
        .status.loading {{
            background: #fff3cd;
            color: #856404;
            border-left: 4px solid #ffc107;
        }}
        .status.connected {{
            background: #d4edda;
            color: #155724;
            border-left: 4px solid #28a745;
        }}
        .status.error {{
            background: #f8d7da;
            color: #721c24;
            border-left: 4px solid #dc3545;
        }}
        .spinner {{
            width: 20px;
            height: 20px;
            border: 3px solid #f3f3f3;
            border-top: 3px solid #856404;
            border-radius: 50%;
            animation: spin 1s linear infinite;
        }}
        @keyframes spin {{
            0% {{ transform: rotate(0deg); }}
            100% {{ transform: rotate(360deg); }}
        }}
        .viewer-container {{
            background: white;
            border-radius: 10px;
            padding: 20px;
            box-shadow: 0 4px 6px rgba(0,0,0,0.1);
            margin-bottom: 20px;
        }}
        #dcv-display {{
            width: 100%;
            height: 800px;
            border: 2px solid #e0e0e0;
            border-radius: 8px;
            background: #000;
            /* Changed from overflow: hidden to overflow: auto to enable scrolling */
            overflow: auto;
            position: relative;
        }}
        .info-panel {{
            background: white;
            padding: 20px;
            border-radius: 10px;
            box-shadow: 0 4px 6px rgba(0,0,0,0.1);
        }}
        .info-panel h3 {{
            color: #333;
            margin-bottom: 15px;
            font-size: 18px;
        }}
        .info-item {{
            display: flex;
            align-items: center;
            gap: 10px;
            padding: 10px;
            background: #f8f9fa;
            border-radius: 5px;
            margin-bottom: 10px;
        }}
        .info-item strong {{
            color: #667eea;
            min-width: 120px;
        }}
        .badge {{
            display: inline-block;
            padding: 4px 12px;
            border-radius: 12px;
            font-size: 12px;
            font-weight: 600;
            background: #667eea;
            color: white;
        }}
        .resolution-control {{
            display: flex;
            align-items: center;
            gap: 15px;
            padding: 10px 15px;
            background-color: #f9f9f9;
            border-radius: 8px;
            border: 1px solid #e1e1e1;
            margin-bottom: 15px;
        }}
        .resolution-label {{
            font-size: 14px;
            font-weight: 500;
            color: #333;
            white-space: nowrap;
        }}
        .resolution-options {{
            display: flex;
            gap: 8px;
            flex-wrap: wrap;
        }}
        .resolution-button {{
            padding: 6px 12px;
            border-radius: 4px;
            font-size: 13px;
            font-weight: 500;
            cursor: pointer;
            border: 1px solid #ccc;
            background: #fff;
            transition: all 0.2s ease;
        }}
        .resolution-button:hover {{
            border-color: #667eea;
        }}
        .resolution-button.active {{
            background: #667eea;
            color: white;
            border-color: #667eea;
        }}
        .resolution-info {{
            font-size: 12px;
            color: #666;
            margin-left: auto;
            white-space: nowrap;
        }}
        .control-panel {{
            display: flex;
            align-items: center;
            gap: 15px;
            padding: 10px 15px;
            background-color: #f9f9f9;
            border-radius: 8px;
            border: 1px solid #e1e1e1;
            margin-bottom: 15px;
        }}
        .control-label {{
            font-size: 14px;
            font-weight: 500;
            color: #333;
            white-space: nowrap;
        }}
        .control-button {{
            padding: 6px 16px;
            border-radius: 4px;
            font-size: 13px;
            font-weight: 500;
            cursor: pointer;
            border: 1px solid #ccc;
            transition: all 0.2s ease;
        }}
        .btn-take-control {{
            background: #28a745;
            color: white;
            border-color: #28a745;
        }}
        .btn-take-control:hover {{
            background: #218838;
            border-color: #1e7e34;
        }}
        .btn-give-control {{
            background: #dc3545;
            color: white;
            border-color: #dc3545;
        }}
        .btn-give-control:hover {{
            background: #c82333;
            border-color: #bd2130;
        }}
        .control-status {{
            font-size: 12px;
            color: #666;
            margin-left: auto;
        }}
    </style>
</head>
<body>
    <div class="container">
        <div class="header">
            <h1>🖥️ Browser Live View</h1>
            <div class="subtitle">
                Real-time streaming powered by AWS AgentCore & Amazon DCV
            </div>
        </div>

        <div id="status" class="status loading">
            <div class="spinner"></div>
            <span>Loading DCV SDK and connecting...</span>
        </div>

        <div class="viewer-container">
            <!-- Control Panel -->
            <div class="control-panel">
                <span class="control-label">Browser Control:</span>
                <button id="take-control-btn" class="control-button btn-take-control">
                    🎮 Human Take Over
                </button>
                <button id="give-control-btn" class="control-button btn-give-control" style="display:none">
                    🤖 Give Back Control
                </button>
                <span class="control-status" id="control-status">Automation Active</span>
            </div>
            
            <!-- Resolution Control -->
            <div class="resolution-control">
                <span class="resolution-label">Display Size:</span>
                <div class="resolution-options" id="resolution-buttons">
                    <button class="resolution-button" data-resolution="720p" title="1280×720">720p</button>
                    <button class="resolution-button active" data-resolution="900p" title="1600×900">900p</button>
                    <button class="resolution-button" data-resolution="1080p" title="1920×1080">1080p</button>
                    <button class="resolution-button" data-resolution="1440p" title="2560×1440">1440p</button>
                </div>
                <span class="resolution-info" id="current-resolution">Current: 1600×900</span>
            </div>

            <div id="dcv-display"></div>
        </div>

        <div class="info-panel">
            <h3>ℹ️ Session Information</h3>
            <div class="info-item">
                <strong>Session ID:</strong>
                <span>{session_id}</span>
            </div>
            <div class="info-item">
                <strong>Streaming Protocol:</strong>
                <span>Amazon DCV (NICE DCV)</span>
            </div>
            <div class="info-item">
                <strong>Browser Backend:</strong>
                <span>AWS Bedrock AgentCore</span>
            </div>
            <div class="info-item">
                <strong>Status:</strong>
                <span id="connection-status" class="badge">Connecting...</span>
            </div>
        </div>
    </div>

    <script>
        // Auto-injected configuration
        const CONFIG = {{
            presignedUrl: '{presigned_url}',
            sessionId: '{session_id}',
            authToken: '{session_id}',
            controlApiUrl: '{CONTROL_API_URL}'  // Injected control API URL
        }};

        // Resolution settings
        const RESOLUTIONS = {{
            '720p': {{ width: 1280, height: 720 }},
            '900p': {{ width: 1600, height: 900 }},
            '1080p': {{ width: 1920, height: 1080 }},
            '1440p': {{ width: 2560, height: 1440 }}
        }};
        let currentResolution = '900p'; // Default resolution
        let hasControl = false; // Track control state

        const statusDiv = document.getElementById('status');
        const statusBadge = document.getElementById('connection-status');
        const spinnerDiv = statusDiv.querySelector('.spinner');
        const resolutionInfo = document.getElementById('current-resolution');
        const resolutionButtons = document.getElementById('resolution-buttons').querySelectorAll('.resolution-button');
        const takeControlBtn = document.getElementById('take-control-btn');
        const giveControlBtn = document.getElementById('give-control-btn');
        const controlStatus = document.getElementById('control-status');
        let dcvConnection = null;

        // Initialize resolution buttons
        resolutionButtons.forEach(button => {{
            button.addEventListener('click', () => {{
                const resolution = button.dataset.resolution;
                changeResolution(resolution);
            }});
        }});

        // Initialize control buttons
        takeControlBtn.addEventListener('click', takeControl);
        giveControlBtn.addEventListener('click', giveBackControl);

        // Take control function
        async function takeControl() {{
            try {{
                updateStatus('Taking control of browser...', 'loading', 'Taking Control');
                
                const response = await fetch(`${{CONFIG.controlApiUrl}}/api/update_browser_stream`, {{
                    method: 'POST',
                    headers: {{
                        'Content-Type': 'application/json'
                    }},
                    body: JSON.stringify({{
                        automationStreamUpdate: {{
                            streamStatus: 'disabled'
                        }}
                    }})
                }});
                
                if (response.ok) {{
                    const result = await response.json();
                    hasControl = true;
                    takeControlBtn.style.display = 'none';
                    giveControlBtn.style.display = 'inline-block';
                    controlStatus.textContent = '🎮 Human Control Active';
                    updateStatus('✅ You now have control of the browser', 'connected', 'Human Control');
                    console.log('Successfully took control of browser:', result);
                }} else {{
                    const errorText = await response.text();
                    updateStatus('❌ Failed to take control of browser', 'error', 'Error');
                    console.error('Failed to take control:', errorText);
                }}
            }} catch (error) {{
                console.error('Error taking control:', error);
                updateStatus(`❌ Error: ${{error.message}}`, 'error', 'Error');
            }}
        }}

        // Give back control function
        async function giveBackControl() {{
            try {{
                updateStatus('Giving back control...', 'loading', 'Releasing Control');
                
                const response = await fetch(`${{CONFIG.controlApiUrl}}/api/update_browser_stream`, {{
                    method: 'POST',
                    headers: {{
                        'Content-Type': 'application/json'
                    }},
                    body: JSON.stringify({{
                        automationStreamUpdate: {{
                            streamStatus: 'enabled'
                        }}
                    }})
                }});
                
                if (response.ok) {{
                    const result = await response.json();
                    hasControl = false;
                    takeControlBtn.style.display = 'inline-block';
                    giveControlBtn.style.display = 'none';
                    controlStatus.textContent = 'Automation Active';
                    updateStatus('✅ Returned control to automation', 'connected', 'Automation Active');
                    console.log('Successfully gave back control:', result);
                }} else {{
                    const errorText = await response.text();
                    updateStatus('❌ Failed to give back control', 'error', 'Error');
                    console.error('Failed to give back control:', errorText);
                }}
            }} catch (error) {{
                console.error('Error giving back control:', error);
                updateStatus(`❌ Error: ${{error.message}}`, 'error', 'Error');
            }}
        }}

        function changeResolution(resolutionId) {{
            // Update UI first
            currentResolution = resolutionId;

            // Update buttons
            resolutionButtons.forEach(btn => {{
                btn.classList.toggle('active', btn.dataset.resolution === resolutionId);
            }});

            // Update info text
            const resolution = RESOLUTIONS[resolutionId];
            resolutionInfo.textContent = `Current: ${{resolution.width}}×${{resolution.height}}`;

            // Update DCV display layout if connection is established
            if (dcvConnection && dcvConnection.requestDisplayLayout) {{
                console.log(`Changing resolution to: ${{resolution.width}}×${{resolution.height}}`);

                dcvConnection.requestDisplayLayout([{{
                    name: "Main Display",
                    rect: {{
                        x: 0,
                        y: 0,
                        width: resolution.width,
                        height: resolution.height
                    }},
                    primary: true
                }}]);

                // Adjust container aspect ratio for better viewing
                const container = document.getElementById('dcv-display');
                if (container) {{
                    const aspectRatio = resolution.height / resolution.width;
                    const containerWidth = container.clientWidth;
                    const newHeight = Math.min(containerWidth * aspectRatio, 800);
                    container.style.height = `${{newHeight}}px`;
                }}
            }}
        }}

        function updateStatus(message, type = 'loading', badgeText = null) {{
            const span = statusDiv.querySelector('span');
            span.textContent = message;
            statusDiv.className = `status ${{type}}`;

            if (type === 'loading') {{
                spinnerDiv.style.display = 'block';
            }} else {{
                spinnerDiv.style.display = 'none';
            }}

            if (badgeText && statusBadge) {{
                statusBadge.textContent = badgeText;
                statusBadge.style.background = type === 'connected' ? '#28a745' :
                                              type === 'error' ? '#dc3545' : '#ffc107';
            }}
        }}

        function loadDCVSDK() {{
            return new Promise((resolve, reject) => {{
                updateStatus('📦 Loading DCV SDK...', 'loading', 'Loading SDK');

                const script = document.createElement('script');
                script.src = '/dcv-sdk/dcvjs-umd/dcv.js';

                script.onload = () => {{
                    if (!window.dcv) {{
                        reject(new Error('DCV SDK loaded but window.dcv not available'));
                        return;
                    }}

                    // HOTFIX: Check if setWorkerPath exists before calling
                    // Some DCV SDK versions might not have this function
                    if (typeof window.dcv.setWorkerPath === 'function') {{
                        try {{
                            window.dcv.setWorkerPath(
                                window.location.origin + '/dcv-sdk/dcvjs-umd/dcv/'
                            );
                            console.log('✅ Worker path configured');
                        }} catch (e) {{
                            console.warn('⚠️ Could not set worker path:', e);
                            // Continue anyway, baseUrl in connect() should handle it
                        }}
                    }} else {{
                        console.log('ℹ️ setWorkerPath not available, will use baseUrl in connect()');
                    }}

                    console.log('✅ DCV SDK loaded successfully');
                    resolve();
                }};

                script.onerror = () => {{
                    reject(new Error('Failed to load DCV SDK. Check if dcv-sdk files exist.'));
                }};

                document.head.appendChild(script);
            }});
        }}

        function authenticateWithDCV() {{
            return new Promise((resolve, reject) => {{
                updateStatus('🔐 Authenticating with DCV server...', 'loading', 'Authenticating');

                window.dcv.authenticate(CONFIG.presignedUrl, {{
                    promptCredentials: (authType, callback) => {{
                        // For presigned URLs, credentials are embedded in the URL
                        // No need to prompt user for credentials
                        console.log('📝 Credentials prompt (using presigned URL)');
                        callback(null, null);
                    }},
                    httpExtraSearchParams: (method, url, body) => {{
                        // Extract query parameters from presigned URL and pass them to WebSocket
                        try {{
                            const parsedUrl = new URL(CONFIG.presignedUrl);
                            const searchParams = parsedUrl.searchParams;
                            console.log('📡 Adding auth params to WebSocket request');
                            return searchParams;
                        }} catch (e) {{
                            console.error('Failed to parse presigned URL:', e);
                            return new URLSearchParams();
                        }}
                    }},
                    success: (auth, result) => {{
                        console.log('✅ Authentication successful');
                        const credentials = result[0];
                        resolve({{
                            presignedUrl: CONFIG.presignedUrl,
                            sessionId: credentials.sessionId,
                            authToken: credentials.authToken
                        }});
                    }},
                    error: (auth, error) => {{
                        console.error('❌ Authentication failed:', error);
                        reject(new Error('Authentication failed: ' + JSON.stringify(error)));
                    }}
                }});
            }});
        }}

        function connectToDCV(credentials) {{
            updateStatus('🔌 Connecting to browser session...', 'loading', 'Connecting');

            const resolution = RESOLUTIONS[currentResolution];
            console.log(`Initial resolution: ${{resolution.width}}×${{resolution.height}}`);

            const dcvConfig = {{
                url: credentials.presignedUrl,
                sessionId: credentials.sessionId,
                authToken: credentials.authToken,
                divId: 'dcv-display',
                baseUrl: window.location.origin + '/dcv-sdk/dcvjs-umd',
                observers: {{
                    httpExtraSearchParams: (method, url, body) => {{
                        // Extract query parameters from presigned URL for streaming connection
                        try {{
                            const parsedUrl = new URL(credentials.presignedUrl);
                            const searchParams = parsedUrl.searchParams;
                            console.log('📡 Adding auth params to streaming connection');
                            return searchParams;
                        }} catch (e) {{
                            console.error('Failed to parse presigned URL:', e);
                            return new URLSearchParams();
                        }}
                    }},
                    firstFrame: () => {{
                        console.log('✅ First frame received');
                        updateStatus('✅ Connected! Streaming browser view...', 'connected', 'Live');

                        // Apply initial resolution after connection is established
                        setTimeout(() => {{
                            if (dcvConnection && dcvConnection.requestDisplayLayout) {{
                                dcvConnection.requestDisplayLayout([{{
                                    name: "Main Display",
                                    rect: {{
                                        x: 0,
                                        y: 0,
                                        width: resolution.width,
                                        height: resolution.height
                                    }},
                                    primary: true
                                }}]);
                                console.log(`Applied initial resolution: ${{resolution.width}}×${{resolution.height}}`);
                            }}
                        }}, 1000);
                    }},
                    displayLayout: (serverWidth, serverHeight, heads) => {{
                        console.log(`Display layout changed: ${{serverWidth}}×${{serverHeight}}`);
                    }},
                    error: (error) => {{
                        console.error('❌ Connection error:', error);
                        updateStatus('❌ Connection error: ' + JSON.stringify(error), 'error', 'Error');
                    }},
                    close: (closeInfo) => {{
                        console.log('Connection closed:', closeInfo);
                        updateStatus('⚠️ Connection closed', 'error', 'Disconnected');
                    }}
                }}
            }};

            window.dcv.connect(dcvConfig)
                .then(connection => {{
                    console.log('Connection established');
                    dcvConnection = connection;

                    // Apply initial resolution
                    setTimeout(() => {{
                        if (connection.requestDisplayLayout) {{
                            connection.requestDisplayLayout([{{
                                name: "Main Display",
                                rect: {{
                                    x: 0,
                                    y: 0,
                                    width: resolution.width,
                                    height: resolution.height
                                }},
                                primary: true
                            }}]);
                            console.log(`Applied initial resolution: ${{resolution.width}}×${{resolution.height}}`);
                        }}
                    }}, 500);
                }})
                .catch(error => {{
                    console.error('Connection failed:', error);
                    updateStatus('❌ Connection error: ' + JSON.stringify(error), 'error', 'Error');
                }});
        }}

        async function initialize() {{
            try {{
                console.log('🚀 Starting live view initialization...');
                console.log('Session ID:', CONFIG.sessionId);

                await loadDCVSDK();
                const credentials = await authenticateWithDCV();
                connectToDCV(credentials);

            }} catch (error) {{
                console.error('Initialization failed:', error);
                updateStatus('❌ Error: ' + error.message, 'error', 'Failed');
            }}
        }}

        window.addEventListener('load', initialize);

        window.addEventListener('beforeunload', () => {{
            if (window.dcv && typeof window.dcv.disconnect === 'function') {{
                window.dcv.disconnect();
            }}
        }});
    </script>
</body>
</html>'''

    with open(HTML_FILE, 'w', encoding='utf-8') as f:
        f.write(html_content)
    
    print(f"   ✅ Created {HTML_FILE}")
    print(f"   📍 Presigned URL injected (expires in 5 minutes)")
    print(f"   🔌 Control API URL configured: {CONTROL_API_URL}")
    return True

def verify_file_structure():
    """필요한 모든 파일이 제자리에 있는지 확인"""
    print("\n🔍 Verifying file structure...")
    
    required = [
        (HTML_FILE, "HTML page"),
        ("dcv-sdk/dcvjs-umd/dcv.js", "DCV SDK (UMD)"),
        ("dcv-sdk/dcvjs-umd/dcv", "DCV workers directory"),
    ]
    
    all_good = True
    for path, description in required:
        exists = os.path.exists(path)
        icon = "✓" if exists else "✗"
        status = "Found" if exists else "MISSING"
        print(f"   {icon} {description}: {status}")
        if not exists:
            all_good = False
    
    return all_good

def start_web_server():
    """백그라운드 스레드에서 HTTP 서버 시작"""
    class QuietHandler(http.server.SimpleHTTPRequestHandler):
        def log_message(self, format, *args):
            # 에러만 로깅
            if args[1] != '200':
                super().log_message(format, *args)
    
    handler = QuietHandler
    
    try:
        httpd = socketserver.TCPServer(("", SERVER_PORT), handler)
        
        # 백그라운드 스레드에서 서버 시작
        server_thread = threading.Thread(target=httpd.serve_forever, daemon=True)
        server_thread.start()
        
        return httpd, server_thread
    except OSError as e:
        if e.errno == 48 or e.errno == 98:  # Address already in use
            print(f"   ⚠️  Port {SERVER_PORT} already in use")
            print(f"   ℹ️  Server might already be running")
            return None, None
        raise

# ============================================================================
# 메인 실행
# ============================================================================

print("="*70)
print("🚀 AUTOMATIC LIVE VIEW SETUP")
print("="*70)

try:
    # Step 1: presigned URL로 HTML 생성
    if not create_html_with_url(live_view_url, session_data["sessionId"]):
        raise Exception("Failed to create HTML file")
    
    # Step 2: 파일 구조 확인
    if not verify_file_structure():
        print("\n❌ Missing required files!")
        print("   Please make sure you ran the DCV SDK download step first.")
        print("\n⚠️  Skipping web server setup due to missing files.")
    else:
        print("\n✅ All files ready!")
        
        # Step 3: 웹 서버 시작
        print("\n🌐 Starting web server...")
        httpd, server_thread = start_web_server()
        
        if httpd:
            print(f"   ✅ Server running on port {SERVER_PORT}")
            print(f"   📍 URL: {SERVER_URL}")
            
            # Step 4: 브라우저 열기
            print("\n🌐 Opening browser...")
            time.sleep(1)  # 서버 시작 시간 대기
            
            try:
                webbrowser.open(SERVER_URL)
                print("   ✅ Browser opened!")
            except Exception as e:
                print(f"   ⚠️  Could not auto-open browser: {e}")
                print(f"   ℹ️  Please manually open: {SERVER_URL}")
            
            # 성공 메시지
            print("\n" + "="*70)
            print("✅ LIVE VIEW IS READY!")
            print("="*70)
            print("\n📺 What you should see in the browser:")
            print("   1. Status: Loading... → Authenticating... → Connected!")
            print("   2. Live browser stream appears in the black area")
            print("   3. Human Take Over button to control the browser")
            print("   4. Resolution controls to adjust the display size")
            print("   5. Session information displayed below")
            print("   6. You can scroll within the browser view using mouse wheel")
            print("\n⚠️  IMPORTANT:")
            print("   - Server is running in background")
            print("   - Presigned URL expires in 5 minutes")
            print("   - Control API running on port 8081")
            print("   - To stop server: Click ■ (stop) button or restart kernel")
            print("\n🔗 Access URL: " + SERVER_URL)
            print("="*70)
            
            # 서버 정보를 접근 가능하게 유지
            print("\n💡 Tip: Leave this cell running to keep the server active.")
            
        else:
            print("\n⚠️  Server might already be running")
            print(f"   Try opening: {SERVER_URL}")
        
except Exception as e:
    print(f"\n❌ Setup failed: {e}")
    import traceback
    traceback.print_exc()
    print("\n💡 Troubleshooting:")
    print("   1. Make sure you ran Step 0 and Step 1 first")
    print("   2. Verify DCV SDK is downloaded (check previous cells)")
    print("   3. Check if port 8080 is available")

## 정리: Browser 세션 중지

테스트가 완료되면 불필요한 비용을 피하기 위해 브라우저 세션을 중지하는 것을 잊지 마세요.

In [ ]:
# # 브라우저 세션 중지
if browser_client:
    browser_client.stop()
    print("✅ Browser session stopped")
else:
    print("⚠️ No active browser session to stop")

## 문제 해결

### 일반적인 문제

#### 1. "Failed to load DCV SDK"

**문제:** HTML 페이지가 DCV SDK 파일을 찾을 수 없습니다.

**해결 방법:**
- 프로젝트에 `dcv-sdk/dcvjs-umd/dcv.js`가 존재하는지 확인
- HTTP를 통해 파일을 제공하고 있는지 확인 (HTML을 직접 열지 않음)
- `python -m http.server 8080`을 사용하여 파일 제공

#### 2. "Authentication failed"

**문제:** presigned URL이 유효하지 않거나 만료되었습니다.

**해결 방법:**
- presigned URL은 기본적으로 5분 후에 만료됩니다
- Step 2.1을 다시 실행하여 새로운 URL을 생성하고 Step 3을 실행하여 새 URL을 웹앱에 임베드합니다. 로컬 브라우저 페이지를 새로고침하면 기존 AgentCore 브라우저 세션에 다시 연결됩니다
- 전체 URL을 복사했는지 확인 (매우 깁니다!)

#### 3. 검은 화면 / 비디오 없음

**문제:** DCV는 연결되지만 브라우저를 표시하지 않습니다.

**해결 방법:**
- 브라우저 콘솔에서 오류 확인
- `setWorkerPath`가 올바른지 확인
- `dcv.connect()`에서 `baseUrl` 매개변수가 설정되어 있는지 확인

#### 4. CORS 오류

**문제:** 브라우저가 CORS 정책으로 인해 요청을 차단합니다.

**해결 방법:**
- HTML 파일을 직접 열지 마세요 (file:// 프로토콜)
- 로컬 웹 서버 사용: `python -m http.server 8080`
- 백엔드 API를 사용하는 경우 Flask CORS가 구성되어 있는지 확인

### 도움 받기

여전히 문제가 있는 경우:

1. 브라우저 콘솔(F12)에서 오류 메시지 확인
2. 모든 파일 경로가 올바른지 확인
3. AWS 자격 증명이 구성되어 있는지 확인
4. 해당 리전에서 AgentCore Browser 서비스를 사용할 수 있는지 확인

## 완전한 구현 예제

React, WebSocket 업데이트 및 프로덕션 기능이 포함된 완전한 구현 예제는 다음을 확인하세요:

**GitHub Repository:**
https://github.com/aws-samples/sample-browser-order-automation-agentcore

이 샘플에는 다음이 포함됩니다:
- 완전한 React 애플리케이션
- FastAPI 백엔드
- 실시간 상태 업데이트
- 세션 녹화
- 멀티 에이전트 오케스트레이션

### 검토할 주요 파일

- `frontend/src/components/LiveBrowserViewer.js` - React 컴포넌트 구현
- `backend/services/browser_service.py` - Browser 세션 관리
- `backend/app.py` - API 엔드포인트

## 요약

이 튜토리얼에서 다음을 배웠습니다:

✅ Python에서 AgentCore Browser client 설정

✅ 안전한 스트리밍을 위한 presigned URL 생성

✅ DCVjs SDK 다운로드 및 구성

✅ 실시간으로 브라우저 뷰를 스트리밍하는 HTML 페이지 생성

✅ 브라우저를 수동으로 제어하기 위한 Human Take Over 버튼 추가

✅ 디스플레이 크기를 조정하기 위한 해상도 컨트롤 추가 (720p, 900p, 1080p, 1440p)

✅ 인증 및 연결을 올바르게 처리

✅ automation mode를 전환하기 위한 control API 구현

✅ 일반적인 문제 해결

### 다음 단계

- HTML 페이지에 더 많은 인터랙티브 컨트롤 추가
- AI agent와 통합 (Nova Act, Strands)
- FastAPI로 프로덕션 백엔드 구축
- 세션 녹화 구현
- 사용자 인증 추가

**즐거운 개발 되세요! 🚀**